In [ ]:
# Important Packages
!pip install nest-asyncio
!git clone --depth=1 https://github.com/twintproject/twint.git
!cd twint && pip3 install . -r requirements.txt
!pip install langdetect
!pip install -q transformers

In [2]:
# Dependencies & Libraries
import nest_asyncio
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from langdetect import detect
import os
import re

BASEDIR = os.getcwd()
print(BASEDIR)
nest_asyncio.apply()


/Users/rui/Documents/GitHub/sentic-ai-project/rnn_bot


In [3]:
# Search Parameters
today = datetime.today().strftime("%Y-%m-%d")
tomorrow = (datetime.today() + timedelta(1)).strftime("%Y-%m-%d")
past_seven_days = (datetime.today() - timedelta(7)).strftime("%Y-%m-%d")

print(f"Getting tweets ranging from\n{today} to {tomorrow}")

keywords = ["BTC", "BITCOIN", "BTC-USD"]

Getting tweets ranging from
2021-10-01 to 2021-10-02


In [4]:
# Data Collection
import twint

# tweets = pd.DataFrame(columns=["date", "user_id", "username", "tweet", "nlikes", "nretweets", "nreplies"])
tweets = pd.DataFrame(columns=["date","tweet"])
c = twint.Config()
# c.Username = "elonmusk"
# c.Limit = 10
c.Limit = 10000
c.Store_csv = False
c.Pandas = True
c.Hide_output = True
c.Since = today
c.Until = tomorrow
c.Min_likes = 10
# Getting only English tweets, undefined == all, 
# Lang codes here https://github.com/twintproject/twint/wiki/Langauge-codes
# This shit is not accurate
c.Lang = "en"

c.Search = "btc OR bitcoin"

twint.run.Search(c)

df = twint.storage.panda.Tweets_df
try:
    #tweets = tweets.append(df[["date", "user_id", "username", "tweet", "nlikes", "nretweets", "nreplies"]])
    tweets = tweets.append(df[["date","tweet"]])
except KeyError:
    print("No more data!")

tweets.to_csv(f"{BASEDIR}/data/tweets.csv")
tweets.head()

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


,date,tweet
0,2021-10-01 17:29:29,#BTC zazaz knows 💡
1,2021-10-01 17:29:13,$BTC Has broken out. Flip 45.2K and we're read...
2,2021-10-01 17:28:50,Bitcoin de kırılım geldi. Pump işi tamam. Önü...
3,2021-10-01 17:28:08,$BTC 45K in 3...2...1... https://t.co/kQuSngn7ND
4,2021-10-01 17:26:56,#Bitcoin Legendary accuracy ✅


In [5]:
# Cleanining Functions
def clean(tweet):
    try:
        if detect(tweet) != "en":
            return np.NaN
    except:
        return np.NaN
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
    tweet = re.sub(r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)", "", tweet)
    tweet = re.sub(r"www.\S+", "", tweet)
    return tweet

In [6]:
tweets["cleaned_tweet"] = tweets["tweet"].apply(clean)
print(tweets.head())
tweets = tweets.dropna()
tweets = tweets.drop_duplicates()
tweets.to_csv(f"{BASEDIR}/data/tweets_cleaned.csv")

                  date                                              tweet  \
0  2021-10-01 17:29:29                                 #BTC zazaz knows 💡   
1  2021-10-01 17:29:13  $BTC Has broken out. Flip 45.2K and we're read...   
2  2021-10-01 17:28:50  Bitcoin de kırılım geldi. Pump işi tamam.  Önü...   
3  2021-10-01 17:28:08  $BTC 45K in 3...2...1...  https://t.co/kQuSngn7ND   
4  2021-10-01 17:26:56                      #Bitcoin Legendary accuracy ✅   

                                       cleaned_tweet  
0                                                NaN  
1  $BTC Has broken out. Flip 45.2K and we're read...  
2                                                NaN  
3                                                NaN  
4                               Legendary accuracy ✅  


In [7]:
# Sentiment Analysis
from transformers import pipeline

sentiment = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Downloading: 100%|██████████| 255M/255M [00:04<00:00, 54.0MB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 18.0kB/s]
Downloading: 100%|██████████| 226k/226k [00:01<00:00, 188kB/s]


In [8]:
def sentiment_analysis(tweet):
    label, score = sentiment(tweet)[0].values()
    if label == "NEGATIVE":
        return 1 - score
    else:
        return score

tweets["sentiment"] = tweets["tweet"].apply(sentiment_analysis)
tweets.head()
tweets.to_csv(f"{BASEDIR}/data/tweets_sentiment.csv")

In [9]:
# Average Sentiment Score
print("Average Sentiment:", tweets["sentiment"].sum()/len(tweets.index))


Average Sentiment: 0.21599949279693045
